In [ ]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json


In [2]:
consumer_key='ddeVX8iF2ZD6FrzMyyBnj8RTC'
consumer_secret='MrCduq8EDGyrrZOIrnpUJZlDs8xSGRamgBW9dOB11hkx7JRwSB'
access_token ='1480887625972011008-ed6bE2J55LEocK7RMGs6vy3irU8yNU'
access_secret='fyPLEViPIRX64TIenSGEkYaKcbtdpMxy9lHLjHZNI1WOy'

In [3]:
class TweetsListener(StreamListener):
    #intead of tweepy.Stream we can use StreamListener which is needed to be imported from tweepy.streaming
  # tweet object listens for the tweets
  def __init__(self, csocket):
    self.client_socket = csocket
    print("new message")
    
  def on_data(self, data):
    try:
     
      msg = json.loads( data )
      print(msg)
      
      # if tweet is longer than 140 characters
      if "extended_tweet" in msg:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['extended_tweet']['full_text']+"t_end")\
            .encode('utf-8'))         
        print(msg['extended_tweet']['full_text'])
      else:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['text']+"t_end")\
            .encode('utf-8'))
        print(msg['text'])
      return True
    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True
  def on_error(self, status):
    print(status)
    return True

In [4]:
def sendData(c_socket, keyword):
  print('start sending data from Twitter to socket')
  # authentication based on the credentials
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  # start sending data from the Streaming API 
  twitter_stream = Stream(auth, TweetsListener(c_socket))
  twitter_stream.filter(track = keyword, languages=["en"])

In [ ]:
if __name__ == "__main__":
    # server (local machine) creates listening socket
    s = socket.socket()
    host = "0.0.0.0"    
    port = 5555
    s.bind((host, port))
    print('socket is ready')
    # server (local machine) listens for connections
    s.listen(4)
    print('socket is listening')
    # return the socket and the address on the other side of the connection (client side)
    c_socket, addr = s.accept()
    print("Received request from: " + str(addr))
    # select here the keyword for the tweet data
    sendData(c_socket, keyword = ['bitcoin'])

socket is ready
socket is listening
